## SeqNeo
- Stage: Cambrian
- Version: Pomoria

#### Related Commands
- tensorboard --logdir runs
- nohup jupyter lab > jupyter.log &

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import copy
import random
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [3]:
from tyrell import spec as S
from tyrell import dsl as D
from tyrell.decider import Example

# Morpheus Version
from MorpheusInterpreter import *
from ProgramSpace import *

In [4]:
torch.__version__

'1.0.0'

In [5]:
class SeqNeo(nn.Module):
    def __init__(self, p_config=None):
        super(SeqNeo, self).__init__()
        self.config = p_config
        
        self.fn_embedding = nn.Embedding(
            self.config["fn_vocab_size"],
            self.config["fn_dim"],
            padding_idx=0,
        )
        
        self.encoder1 = nn.Linear(
            self.config["encoder"]["input_dim"],
            2048,
        )
        self.encoder2 = nn.Linear(
            2048,
            self.config["encoder"]["output_dim"],
        )
        
        self.decoder = nn.GRUCell(
            self.config["decoder"]["input_dim"],
            self.config["decoder"]["hidden_dim"],
            bias=True,
        )
        self.classifier = nn.Linear(
            self.config["decoder"]["hidden_dim"],
            self.config["decoder"]["output_dim"],
        )
        
    def decode(self, pin_exp, pout_exp, p_hidden, pfn_context):
        tmp_exp = self.encode(pin_exp, pout_exp) # (B=1, encoder_output_dim)
        tmp_fn = self.fn_embedding(pfn_context) # (B=1, fn_dim)
        tmp_input = torch.cat([tmp_exp,tmp_fn],dim=1)
        
        if p_hidden is None:
            # very first
            tmp_hidden = self.decoder(tmp_input)
        else:
            # not the very first
            tmp_hidden = self.decoder(tmp_input, p_hidden)
        # (B=1, decoder_hidden_dim)
        
        tmp_output = torch.log_softmax(
            self.classifier(tmp_hidden),
            dim=1
        ) # (B=1, decoder_output_dim)
        
        # first hidden, then output
        return (tmp_hidden, tmp_output)
        
        
    def encode(self, pin_exp, pout_exp):
        # pin_exp/pout_exp: (B=1, abs_dim)
        tmp_exp = torch.cat([pin_exp,pout_exp],dim=1) # (B=1, encoder_input_dim=abs_dim*2)
        tmp_out1 = F.relu(self.encoder1(tmp_exp))
        tmp_out2 = F.relu(self.encoder2(tmp_out1))
        return tmp_out2 # (B=1, encoder_output_dim)
    

In [6]:
# replace certain node id with certain value
def modify_shell(p_shell, p_id_from, p_id_to):
    d_prod = p_shell[0]
    d_rhs = p_shell[1]
    ld_rhs = [p_id_to if d_rhs[i]==p_id_from else d_rhs[i]
             for i in range(len(d_rhs))]
    return (d_prod, tuple(ld_rhs))

# using default path and solutions
from benchmarks.pldi17_sol import *
def load_benchmark(p_id, p_sourceps, p_builder):
    d_prog = p_builder._from_sexp(solutions[p_id])
    d_input = p_sourceps.interpreter.init_tbl("./benchmarks/pldi17/p{}_input1.csv".format(p_id))
    p_sourceps.interpreter.create_shadow(d_input)
    d_output = p_sourceps.interpreter.init_tbl("./benchmarks/pldi17/p{}_output1.csv".format(p_id))
    p_sourceps.interpreter.create_shadow(d_output)
    d_ps = ProgramSpace(
        p_sourceps.spec, p_sourceps.interpreter, [d_input], d_output,
    )
    d_ps.init_by_prog(d_prog)
    return d_ps

In [7]:
def SeqBeam(p_config, p_sourceps, p_model, p_dataset):
    print("# Start Beam Search...")
    p_builder = D.Builder(p_sourceps.spec)
    
    for d_episode in range(p_config["train"]["n_episode"]):
        p_model.eval()

        if isinstance(p_dataset,list):
            print("# sample from dataset #")
            # ====== option 1: sample from dataset ====== #
            eid = random.choice(range(len(p_dataset)))
            data_prog, data_str_example = p_dataset[eid]
            data_example = Example(
                input=[p_sourceps.interpreter.load_data_into_var(p) for p in data_str_example.input],
                output=p_sourceps.interpreter.load_data_into_var(data_str_example.output),
            )
            ps_solution = ProgramSpace(
                p_sourceps.spec, p_sourceps.interpreter, data_example.input, data_example.output,
            )
            ps_solution.init_by_prog(data_prog)
        elif isinstance(p_dataset,int):
            print("# load Morpheus benchmark #")
            # ====== option 2: load Morpheus benchmarks ====== #
            ps_solution = load_benchmark(p_dataset, p_sourceps, p_builder)
        else:
            print("# sample from generator #")
            # ====== option 3: sample from generator ====== #
            ps_solution = p_dataset.get_new_chain_program(
                p_config["train"]["n_size"] + 1 # depth=size+1
            )

        # solution self-check
        if ps_solution.check_eq() is None:
            continue
            print("ERROR, SOLUTION NOT CONSISTENT!")
        else:
            print("==== Problem ====")
            print(str(ps_solution.node_list[-1]))

        selected_neurons = []

        # initialize a new Program Space
        ps_current = ProgramSpace(
            p_sourceps.spec, p_sourceps.interpreter, ps_solution.inputs, ps_solution.output,
        )
        # then initialize a shell template
        tmp_shell_list = ps_current.get_neighboring_shells()
        tmp_node_to_replace = ps_current.node_dict["ParamNode"][0] # for chain only
        # replace the Param Node id in shells with -1 to make them templates
        template_list = [
            modify_shell(tmp_shell_list[i],tmp_node_to_replace,-1)
            for i in range(len(tmp_shell_list))
        ]
        
        beam_list = [
            # (score, hidden, fn_context, ProgramSpace)
            (0.0, None, 0, ps_current.make_copy())
        ]

        for d_step in range(len(ps_solution.shells)):
            sprout_list = []
            for d_bid in range(len(beam_list)):
                dd_score, dd_hidden, dd_context, dd_ps = beam_list[d_bid]
                ps_current = dd_ps
                
                # ### assume chain execution, so only 1 possible returns
                # ### at d_step=0, this should be input[0]
                id_current = ps_current.get_strict_frontiers()[0]
                var_current = ps_current.node_list[id_current].ps_data # need the real var name in r env
                var_output = ps_current.output

                map_current = p_sourceps.interpreter.camb_get_ventogyrus(var_current)
                map_output = p_sourceps.interpreter.camb_get_ventogyrus(var_output)

                # make current shell list
                current_shell_list = [
                    modify_shell(template_list[i],-1,id_current)
                    for i in range(len(template_list))
                ]
                current_shell_list = [None] + current_shell_list # None for <SOS> in nn embedding

                # wrap in B=1
                if use_cuda:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float)).cuda()
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float)).cuda()
                    td_context = Variable(torch.tensor([dd_context],dtype=torch.long)).cuda()
                else:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float))
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float))
                    td_context = Variable(torch.tensor([dd_context],dtype=torch.long))

                # (B=1, fn_vocab_size)
                hidden_current, td_pred = p_model.decode(td_current, td_output, dd_hidden, td_context)
                
                # start sprouting
                np_pred = td_pred.data.cpu().numpy().flatten()
                as_pred = np.argsort(np_pred)[::-1]
                for i in range(p_config["test"]["sprout_size"]):
                    ppps = ps_current.make_copy()
                    tmp_id = as_pred[i]
                    fn_context = tmp_id
                    # try to update
                    update_status = ppps.add_neighboring_shell(
                        current_shell_list[tmp_id]
                    )
                    if update_status:
                        # success, put into sprout list
                        sprout_list.append(
                            (
                                dd_score+np_pred[tmp_id],
                                hidden_current,
                                fn_context,
                                ppps,
                            )
                        )
                    # else: do nothing
                # <END_FOR_SPROUT>
            # <END_FOR_BEAM>
            sorted_sprout_list = sorted(
                sprout_list,
                key=lambda x:x[0],
                reverse=True,
            )
            if len(sorted_sprout_list)>p_config["test"]["beam_size"]:
                beam_list = sorted_sprout_list[:p_config["test"]["beam_size"]]
            else:
                beam_list = sorted_sprout_list
        # <END_FOR_STEP>
        
        # then print and seek for correct solutions
        print("==== beam search ====")
        for i in range(len(beam_list)):
            dd_score, dd_hidden, dd_context, dd_ps = beam_list[i]
            print("#{}{}, {:.4f}, {}".format(
                "(solved)" if dd_ps.check_eq() is not None else "",
                i+1,
                dd_score,
                str(dd_ps.node_list[-1])
            ))
        
        input("==== Press to enter next problem ====")

    # <END_FOR_EPISODE>
    

In [8]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb6.tyrell')
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
)
# dumb variable to help infer the shells
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None,
)

In [9]:
m_config = {
    "abs_dim": 15*7+1,
    "fn_dim": 128,
    "fn_vocab_size": len(m_ps.get_neighboring_shells())+1, # with one more <SOS>
    "encoder":{
        "input_dim": None,
        "output_dim": 1024, # hidden state
    },
    "decoder":{
        "input_dim": None,
        "hidden_dim": None,
        "output_dim": None,
    },
    "train":{
        "n_epoch":500, # #epoches in total
        "n_episode": 100, # #problems per epoch
        "n_size": 3,
    },
    "test":{
        "beam_size": 40,
        "sprout_size": 40,
    },
}
m_config["encoder"]["input_dim"] = m_config["abs_dim"]*2
m_config["decoder"]["input_dim"] = \
    m_config["encoder"]["output_dim"] + \
    m_config["fn_dim"]
m_config["decoder"]["hidden_dim"] = m_config["encoder"]["output_dim"]
m_config["decoder"]["output_dim"] = m_config["fn_vocab_size"]

In [15]:
seq_neo = SeqNeo(p_config=m_config)
seq_neo.load_state_dict(torch.load("./saved_models/0811CambAgent_Thomas_Size3_ep30.pt"))
if use_cuda:
    seq_neo = seq_neo.cuda()

In [11]:
# SeqBeam(m_config, m_ps, seq_neo, m_dataset)

In [12]:
SeqBeam(m_config, m_ps, seq_neo, 22)

# Start Beam Search...
# load Morpheus benchmark #
==== Problem ====
summarise(group_by(gather(@param0, ['4', '5']), ['4']), mean, 5)
==== beam search ====
#1, -4.5169, summarise(group_by(neg_gather(@param0, ['1', '2']), ['2']), mean, 4)
#2, -4.9277, summarise(group_by(neg_gather(@param0, ['1', '2']), ['3']), mean, 4)
#3, -5.4751, summarise(group_by(neg_gather(@param0, ['1', '2']), ['2']), sum, 4)
#4, -5.5633, summarise(group_by(neg_gather(@param0, ['1', '2']), ['3']), sum, 4)
#5, -6.2637, summarise(group_by(neg_gather(@param0, ['1', '2']), ['4']), mean, 4)
#6, -6.2735, summarise(group_by(neg_gather(@param0, ['1', '2']), ['1']), sum, 4)
#7, -6.4643, summarise(group_by(neg_gather(@param0, ['1', '2']), ['1']), mean, 4)
#8, -6.8272, summarise(group_by(neg_gather(@param0, ['1', '2']), ['4']), sum, 4)
#9, -7.0451, summarise(group_by(gather(@param0, ['4', '5']), ['3']), min, 3)
#10, -7.0797, group_by(group_by(neg_gather(@param0, ['1', '2']), ['3']), ['2'])
#11, -7.1711, summarise(group_by(ga

KeyboardInterrupt: 

In [ ]:
SeqBeam(m_config, m_ps, seq_neo, 56)

# Start Beam Search...
# load Morpheus benchmark #
==== Problem ====
spread(unite(neg_gather(@param0, ['1', '2']), 1, 3), 1, 3)
==== beam search ====
#1, -5.5511, summarise(group_by(gather(@param0, ['1', '2']), ['2']), min, 1)
#2, -5.5747, summarise(group_by(gather(@param0, ['3', '4']), ['1']), mean, 3)
#3, -6.1262, summarise(group_by(gather(@param0, ['3', '4']), ['1']), max, 3)
#4, -6.1536, summarise(group_by(gather(@param0, ['4', '5']), ['2']), min, 3)
#5, -6.2172, summarise(group_by(gather(@param0, ['4', '5']), ['2']), max, 3)
#6, -6.2984, summarise(group_by(neg_gather(@param0, ['1', '2']), ['4']), sum, 4)
#7, -6.4296, summarise(group_by(gather(@param0, ['3', '4']), ['3', '5']), max, 3)
#8, -6.5212, summarise(group_by(gather(@param0, ['4', '5']), ['2']), mean, 3)
#9, -6.7199, summarise(group_by(gather(@param0, ['3', '4']), ['5']), mean, 3)
#10, -6.7678, summarise(group_by(neg_gather(@param0, ['1', '2']), ['2']), max, 4)
#11, -6.8255, summarise(group_by(gather(@param0, ['4', '5']), [

In [14]:
SeqBeam(m_config, m_ps, seq_neo, 2)

# Start Beam Search...
# load Morpheus benchmark #
==== Problem ====
spread(unite(gather(@param0, ['3', '4']), 2, 3), 2, 3)
==== beam search ====
#1, -5.8201, summarise(group_by(neg_gather(@param0, ['3', '4']), ['3']), sum, 1)
#2, -5.9246, summarise(group_by(neg_gather(@param0, ['3', '4']), ['2']), sum, 1)
#3, -6.1348, summarise(group_by(neg_gather(@param0, ['1', '2']), ['2']), mean, 4)
#4, -6.2790, summarise(group_by(neg_gather(@param0, ['3', '4']), ['2']), min, 1)
#5, -6.2955, summarise(group_by(neg_gather(@param0, ['3', '4']), ['3']), min, 1)
#6, -6.3906, summarise(group_by(neg_gather(@param0, ['3', '4']), ['4']), sum, 1)
#7, -6.7723, summarise(group_by(neg_gather(@param0, ['3', '4']), ['3']), mean, 1)
#8, -6.8054, summarise(group_by(neg_gather(@param0, ['1', '2']), ['4']), mean, 4)
#9, -6.8337, summarise(group_by(neg_gather(@param0, ['1', '2']), ['3']), mean, 4)
#10, -6.8734, summarise(group_by(neg_gather(@param0, ['3', '4']), ['3']), min, 2)
#11, -6.9161, summarise(group_by(neg_ga

KeyboardInterrupt: 